# Setup 

In [2]:
# Setup - Run only once per Kernel App
%conda install openjdk -y

# install PySpark
%pip install pyspark==3.3.0

# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.3.1
  latest version: 23.9.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.9.0



# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Import pyspark and build Spark session
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.appName("PySparkApp")
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.2.2")
    .config(
        "fs.s3a.aws.credentials.provider",
        "com.amazonaws.auth.ContainerCredentialsProvider",
    )
    .getOrCreate()
)

print(spark.version)

:: loading settings :: url = jar:file:/opt/conda/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e5c18e9e-3599-48d1-aff2-57bc9b53157f;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.563 in central
:: resolution report :: resolve 469ms :: artifacts dl 39ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.563 from central in [default]
	org.apache.hadoop#hadoop-aws;3.2.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	---------------------------------------------

23/11/06 17:13:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


3.3.0


In [4]:
import sagemaker

session = sagemaker.Session()
bucket = session.default_bucket()
print(bucket)

# Create or retrieve a Spark session
spark = SparkSession.builder.appName("ReadS3Parquet").getOrCreate()


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker-us-east-1-711387073580
23/11/06 17:13:54 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [5]:
from pyspark.sql.functions import date_format
from pyspark.sql.functions import avg
from pyspark.sql.functions import to_date
from pyspark.sql.functions import date_format
from pyspark.sql.functions import count

# Data

In [6]:
# S3 directory path
s3_directory = f"s3a://{bucket}/project/cleaned/comments/"

# Read all the Parquet files in the directory into a DataFrame
df_comments = spark.read.parquet(s3_directory)

23/11/06 17:16:26 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [7]:
# S3 directory path
s3_directory = f"s3a://{bucket}/project/cleaned/submissions/"

# Read all the Parquet files in the directory into a DataFrame
df_submissions = spark.read.parquet(s3_directory)

23/11/06 17:16:35 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [8]:
# Read in stocks data 
import pandas as pd

df_sp = pd.read_csv("../../data/csv/SP500.csv")
df_dj = pd.read_csv("../../data/csv/DJ.csv")

# Importing Data for other Subreddits 

In [9]:
# Tegveer subreddits
tegveer_bucket = 'sagemaker-us-east-1-433974840707'
s3_directory_tegveer = f"s3a://{tegveer_bucket}/project/cleaned/submissions/"
s3_directory_tegveer_comments = f"s3a://{tegveer_bucket}/project/cleaned/comments/"

# Read all the Parquet files in the directory into a DataFrame
df_submissions_centrist_liberterian = spark.read.parquet(s3_directory_tegveer)
df_comments_centrist_liberterian = spark.read.parquet(s3_directory_tegveer_comments)


In [10]:
# Eric subreddits
eric_bucket = 'sagemaker-us-east-1-395393721134'
s3_directory_eric = f"s3a://{eric_bucket}/project/cleaned/submissions/"
s3_directory_eric_comments = f"s3a://{eric_bucket}/project/cleaned/comments/"

# Read all the Parquet files in the directory into a DataFrame
df_submissions_socialism_economics = spark.read.parquet(s3_directory_eric)
df_submissions_socialism_economics_comments = spark.read.parquet(s3_directory_eric_comments)

In [11]:
# Raunak subreddits

raunak_bucket = 'sagemaker-us-east-1-224518912016'
s3_directory_raunak = f"s3a://{raunak_bucket}/project/cleaned/submissions/"
s3_directory_raunak_comments = f"s3a://{raunak_bucket}/project/cleaned/comments/"

# Read all the Parquet files in the directory into a DataFrame
df_submissions_conservative_finance = spark.read.parquet(s3_directory_raunak)
df_submissions_conservative_finance_comments = spark.read.parquet(s3_directory_raunak_comments)

In [12]:
# My subreddits 

df_submissions_askpol_cmv = df_submissions.filter(df_submissions['subreddit'].isin(['changemyview', 'Ask_Politics']))

# EDA

# Calendar Heatmap

First we need to start by having each of the four political subreddits in a dataframe of its own.

In [13]:
from pyspark.sql.functions import date_format

# Convert created_utc to only have the date
df_submissions_liberterian = df_submissions_centrist_liberterian.withColumn("created_utc", date_format("created_utc", "yyyy-MM-dd")).filter(df_submissions_centrist_liberterian['subreddit'] == 'Libertarian').select(['created_utc', 'subreddit'])
df_submissions_centrist = df_submissions_centrist_liberterian.withColumn("created_utc", date_format("created_utc", "yyyy-MM-dd")).filter(df_submissions_centrist_liberterian['subreddit'] == 'centrist').select(['created_utc', 'subreddit'])
df_submissions_liberal = df_submissions_socialism_economics.withColumn("created_utc", date_format("created_utc", "yyyy-MM-dd")).filter(df_submissions_socialism_economics['subreddit'] == 'Liberal').select(['created_utc', 'subreddit'])
df_submissions_conservative = df_submissions_conservative_finance.withColumn("created_utc", date_format("created_utc", "yyyy-MM-dd")).filter(df_submissions_conservative_finance['subreddit'] == 'Conservative').select(['created_utc', 'subreddit'])
df_submissions_socialism = df_submissions_socialism_economics.withColumn("created_utc", date_format("created_utc", "yyyy-MM-dd")).filter(df_submissions_socialism_economics['subreddit'] == 'socialism').select(['created_utc', 'subreddit'])

In [14]:
# Display the modified dataframe
df_submissions_centrist.show(5)

+-----------+---------+
|created_utc|subreddit|
+-----------+---------+
| 2021-05-09| centrist|
| 2021-08-06| centrist|
| 2021-01-07| centrist|
| 2021-03-13| centrist|
| 2021-06-25| centrist|
+-----------+---------+
only showing top 5 rows



In [13]:
df_submissions_liberterian.show(5)

+-----------+-----------+
|created_utc|  subreddit|
+-----------+-----------+
| 2021-09-06|Libertarian|
| 2021-09-06|Libertarian|
| 2021-06-13|Libertarian|
| 2021-06-13|Libertarian|
| 2021-05-09|Libertarian|
+-----------+-----------+
only showing top 5 rows



In [14]:
df_submissions_conservative.show(5)

+-----------+------------+
|created_utc|   subreddit|
+-----------+------------+
| 2021-04-18|Conservative|
| 2021-04-18|Conservative|
| 2021-04-18|Conservative|
| 2021-04-18|Conservative|
| 2021-04-18|Conservative|
+-----------+------------+
only showing top 5 rows



In [15]:
df_submissions_socialism.show(5)

+-----------+---------+
|created_utc|subreddit|
+-----------+---------+
| 2021-12-22|socialism|
| 2021-12-22|socialism|
| 2021-12-22|socialism|
| 2021-10-12|socialism|
| 2021-10-12|socialism|
+-----------+---------+
only showing top 5 rows



In [21]:
df_submissions_liberal.show(5)

+-----------+---------+
|created_utc|subreddit|
+-----------+---------+
| 2022-06-28|  Liberal|
| 2021-01-18|  Liberal|
| 2021-01-18|  Liberal|
| 2021-03-22|  Liberal|
| 2021-07-29|  Liberal|
+-----------+---------+
only showing top 5 rows



In [17]:
df_submissions_conservative.show()

+-----------+------------+------+-----+
|created_utc|   subreddit|   Day|Month|
+-----------+------------+------+-----+
| 2021-04-18|Conservative|Sunday|April|
| 2021-04-18|Conservative|Sunday|April|
| 2021-04-18|Conservative|Sunday|April|
| 2021-04-18|Conservative|Sunday|April|
| 2021-04-18|Conservative|Sunday|April|
| 2021-04-19|Conservative|Monday|April|
| 2021-04-19|Conservative|Monday|April|
| 2021-04-19|Conservative|Monday|April|
| 2021-04-19|Conservative|Monday|April|
| 2021-04-19|Conservative|Monday|April|
| 2021-04-19|Conservative|Monday|April|
| 2021-04-19|Conservative|Monday|April|
| 2021-04-19|Conservative|Monday|April|
| 2021-04-19|Conservative|Monday|April|
| 2021-04-19|Conservative|Monday|April|
| 2021-04-19|Conservative|Monday|April|
| 2021-04-19|Conservative|Monday|April|
| 2021-04-19|Conservative|Monday|April|
| 2021-04-19|Conservative|Monday|April|
| 2021-04-19|Conservative|Monday|April|
+-----------+------------+------+-----+
only showing top 20 rows



In [24]:
from pyspark.sql.functions import count

df_grouped_conservative = df_submissions_conservative.groupBy("Day", "Month").agg(count("*").alias("Count")).orderBy("Month", "Day")

df_grouped_conservative.show()


+---------+--------+-----+
|      Day|   Month|Count|
+---------+--------+-----+
|   Friday|   April| 4825|
|   Monday|   April| 3795|
| Saturday|   April| 3591|
|   Sunday|   April| 2862|
| Thursday|   April| 4784|
|  Tuesday|   April| 4285|
|Wednesday|   April| 4187|
|   Friday|  August| 3696|
|   Monday|  August| 4301|
| Saturday|  August| 2714|
|   Sunday|  August| 3064|
| Thursday|  August| 3851|
|  Tuesday|  August| 4804|
|Wednesday|  August| 4138|
|   Friday|December| 3364|
|   Monday|December| 2766|
| Saturday|December| 2473|
|   Sunday|December| 2245|
| Thursday|December| 3679|
|  Tuesday|December| 2864|
+---------+--------+-----+
only showing top 20 rows



In [25]:
df_grouped_liberterian = df_submissions_liberterian.groupBy("Day", "Month").agg(count("*").alias("Count")).orderBy("Month", "Day")

df_grouped_liberterian.show()


+---------+--------+-----+
|      Day|   Month|Count|
+---------+--------+-----+
|   Friday|   April|  829|
|   Monday|   April|  604|
| Saturday|   April|  579|
|   Sunday|   April|  476|
| Thursday|   April|  849|
|  Tuesday|   April|  659|
|Wednesday|   April|  694|
|   Friday|  August|  504|
|   Monday|  August|  596|
| Saturday|  August|  376|
|   Sunday|  August|  441|
| Thursday|  August|  492|
|  Tuesday|  August|  608|
|Wednesday|  August|  533|
|   Friday|December|  509|
|   Monday|December|  357|
| Saturday|December|  317|
|   Sunday|December|  391|
| Thursday|December|  488|
|  Tuesday|December|  385|
+---------+--------+-----+
only showing top 20 rows



In [26]:
df_grouped_centrist = df_submissions_centrist.groupBy("Day", "Month").agg(count("*").alias("Count")).orderBy("Month", "Day")

df_grouped_centrist.show()

+---------+--------+-----+
|      Day|   Month|Count|
+---------+--------+-----+
|   Friday|   April|  189|
|   Monday|   April|  138|
| Saturday|   April|  113|
|   Sunday|   April|   82|
| Thursday|   April|  197|
|  Tuesday|   April|  150|
|Wednesday|   April|  145|
|   Friday|  August|  153|
|   Monday|  August|  160|
| Saturday|  August|   86|
|   Sunday|  August|  121|
| Thursday|  August|  179|
|  Tuesday|  August|  185|
|Wednesday|  August|  181|
|   Friday|December|  151|
|   Monday|December|   81|
| Saturday|December|   87|
|   Sunday|December|   83|
| Thursday|December|  132|
|  Tuesday|December|   88|
+---------+--------+-----+
only showing top 20 rows



In [27]:
df_grouped_socialism = df_submissions_socialism.groupBy("Day", "Month").agg(count("*").alias("Count")).orderBy("Month", "Day")

df_grouped_socialism.show()

+---------+--------+-----+
|      Day|   Month|Count|
+---------+--------+-----+
|   Friday|   April|  543|
|   Monday|   April|  418|
| Saturday|   April|  443|
|   Sunday|   April|  413|
| Thursday|   April|  491|
|  Tuesday|   April|  447|
|Wednesday|   April|  459|
|   Friday|  August|  383|
|   Monday|  August|  412|
| Saturday|  August|  350|
|   Sunday|  August|  363|
| Thursday|  August|  359|
|  Tuesday|  August|  459|
|Wednesday|  August|  403|
|   Friday|December|  420|
|   Monday|December|  333|
| Saturday|December|  329|
|   Sunday|December|  310|
| Thursday|December|  444|
|  Tuesday|December|  362|
+---------+--------+-----+
only showing top 20 rows



In [ ]:
df_grouped_liberal = df_submissions_liberal.groupBy("Day", "Month").agg(count("*").alias("Count")).orderBy("Month", "Day")

df_grouped_liberal.show()

+---------+--------+-----+
|      Day|   Month|Count|
+---------+--------+-----+
|   Friday|   April|  152|
|   Monday|   April|  130|
| Saturday|   April|  119|
|   Sunday|   April|  115|
| Thursday|   April|  155|
|  Tuesday|   April|  145|
|Wednesday|   April|  131|
|   Friday|  August|  119|
|   Monday|  August|  134|
| Saturday|  August|   80|
|   Sunday|  August|   86|
| Thursday|  August|   90|
|  Tuesday|  August|  143|
|Wednesday|  August|  127|
|   Friday|December|  108|
|   Monday|December|   85|
| Saturday|December|   71|
|   Sunday|December|  156|
| Thursday|December|  112|
|  Tuesday|December|   71|
+---------+--------+-----+
only showing top 20 rows



Create a calendar heatmap as per: https://www.highcharts.com/demo/highcharts/heatmap

In [22]:
 # Convert to HighChartJS data structure 

day_mapping = {
    'Monday': 0,
    'Tuesday': 1,
    'Wednesday': 2,
    'Thursday': 3,
    'Friday': 4,
    'Saturday': 5,
    'Sunday': 6
}

month_mapping = {
    'January': 0,
    'February': 1,
    'March': 2,
    'April': 3,
    'May': 4,
    'June': 5,
    'July': 6,
    'August': 7,
    'September': 8,
    'October': 9,
    'November': 10,
    'December': 11
}

# Convert dataframe to table data
table_data = [(row['Day'], row['Month'], row['Count']) for row in df_grouped_socialism.collect()]

# Convert the table data to the desired array format:
array_data = [[month_mapping[month], day_mapping[day], count] for day, month, count in table_data]

In [23]:
array_data

[[3, 4, 543],
 [3, 0, 418],
 [3, 5, 443],
 [3, 6, 413],
 [3, 3, 491],
 [3, 1, 447],
 [3, 2, 459],
 [7, 4, 383],
 [7, 0, 412],
 [7, 5, 350],
 [7, 6, 363],
 [7, 3, 359],
 [7, 1, 459],
 [7, 2, 403],
 [11, 4, 420],
 [11, 0, 333],
 [11, 5, 329],
 [11, 6, 310],
 [11, 3, 444],
 [11, 1, 362],
 [11, 2, 580],
 [1, 4, 652],
 [1, 0, 670],
 [1, 5, 628],
 [1, 6, 620],
 [1, 3, 672],
 [1, 1, 702],
 [1, 2, 637],
 [0, 4, 760],
 [0, 0, 774],
 [0, 5, 780],
 [0, 6, 785],
 [0, 3, 775],
 [0, 1, 756],
 [0, 2, 683],
 [6, 4, 477],
 [6, 0, 379],
 [6, 5, 432],
 [6, 6, 419],
 [6, 3, 446],
 [6, 1, 386],
 [6, 2, 371],
 [5, 4, 444],
 [5, 0, 328],
 [5, 5, 396],
 [5, 6, 315],
 [5, 3, 461],
 [5, 1, 439],
 [5, 2, 468],
 [2, 4, 624],
 [2, 0, 615],
 [2, 5, 512],
 [2, 6, 493],
 [2, 3, 684],
 [2, 1, 758],
 [2, 2, 769],
 [4, 4, 392],
 [4, 0, 497],
 [4, 5, 434],
 [4, 6, 518],
 [4, 3, 440],
 [4, 1, 495],
 [4, 2, 461],
 [10, 4, 338],
 [10, 0, 356],
 [10, 5, 314],
 [10, 6, 305],
 [10, 3, 305],
 [10, 1, 457],
 [10, 2, 406],
 [9, 4

In [24]:
# Convert the Spark DataFrame to a list of tuples
table_data = [(row['Day'], row['Month'], row['Count']) for row in df_grouped_socialism.collect()]


In [25]:
def dataframe_to_highchart_array(df):
    """Convert a Spark DataFrame to HighChartJS array format."""
    
    # Mapping for days and months
    day_mapping = {
        'Monday': 0,
        'Tuesday': 1,
        'Wednesday': 2,
        'Thursday': 3,
        'Friday': 4,
        'Saturday': 5,
        'Sunday': 6
    }

    month_mapping = {
        'January': 0,
        'February': 1,
        'March': 2,
        'April': 3,
        'May': 4,
        'June': 5,
        'July': 6,
        'August': 7,
        'September': 8,
        'October': 9,
        'November': 10,
        'December': 11
    }

    # Convert dataframe to table data
    table_data = [(row['Day'], row['Month'], row['Count']) for row in df.collect()]

    # Convert the table data to the desired array format
    array_data = [[month_mapping[month], day_mapping[day], count] for day, month, count in table_data]

    return array_data

# Now, you can use this function for your DataFrames:
array_data_socialism = dataframe_to_highchart_array(df_grouped_socialism)
array_data_centrist = dataframe_to_highchart_array(df_grouped_centrist)
array_data_libertarian = dataframe_to_highchart_array(df_grouped_liberterian)
array_data_conservative = dataframe_to_highchart_array(df_grouped_conservative)


In [26]:
array_data_conservative

[[3, 4, 4825],
 [3, 0, 3795],
 [3, 5, 3591],
 [3, 6, 2862],
 [3, 3, 4784],
 [3, 1, 4285],
 [3, 2, 4187],
 [7, 4, 3696],
 [7, 0, 4301],
 [7, 5, 2714],
 [7, 6, 3064],
 [7, 3, 3851],
 [7, 1, 4804],
 [7, 2, 4138],
 [11, 4, 3364],
 [11, 0, 2766],
 [11, 5, 2473],
 [11, 6, 2245],
 [11, 3, 3679],
 [11, 1, 2864],
 [11, 2, 3453],
 [1, 4, 5768],
 [1, 0, 4962],
 [1, 5, 4680],
 [1, 6, 4437],
 [1, 3, 5902],
 [1, 1, 5416],
 [1, 2, 5742],
 [0, 4, 6604],
 [0, 0, 6224],
 [0, 5, 5686],
 [0, 6, 5488],
 [0, 3, 6646],
 [0, 1, 5793],
 [0, 2, 6562],
 [6, 4, 4549],
 [6, 0, 3371],
 [6, 5, 3718],
 [6, 6, 3056],
 [6, 3, 4222],
 [6, 1, 3657],
 [6, 2, 3730],
 [5, 4, 3864],
 [5, 0, 3424],
 [5, 5, 3203],
 [5, 6, 2811],
 [5, 3, 4164],
 [5, 1, 4297],
 [5, 2, 4713],
 [2, 4, 5872],
 [2, 0, 5123],
 [2, 5, 4255],
 [2, 6, 3863],
 [2, 3, 6350],
 [2, 1, 6693],
 [2, 2, 6652],
 [4, 4, 3828],
 [4, 0, 4111],
 [4, 5, 3419],
 [4, 6, 3383],
 [4, 3, 3949],
 [4, 1, 4487],
 [4, 2, 4252],
 [10, 4, 3474],
 [10, 0, 3614],
 [10, 5, 2766],


# Dependency Wheel

### Common Users Analysis

In [17]:
df_submissions_liberterian_users = df_submissions_centrist_liberterian.withColumn("created_utc", date_format("created_utc", "yyyy-MM-dd")).filter(df_submissions_centrist_liberterian['subreddit'] == 'Libertarian').select(['author'])
df_submissions_centrist_users = df_submissions_centrist_liberterian.withColumn("created_utc", date_format("created_utc", "yyyy-MM-dd")).filter(df_submissions_centrist_liberterian['subreddit'] == 'centrist').select(['author'])
df_submissions_conservative_users = df_submissions_conservative_finance.withColumn("created_utc", date_format("created_utc", "yyyy-MM-dd")).filter(df_submissions_conservative_finance['subreddit'] == 'Conservative').select(['author'])
df_submissions_socialism_users = df_submissions_socialism_economics.withColumn("created_utc", date_format("created_utc", "yyyy-MM-dd")).filter(df_submissions_socialism_economics['subreddit'] == 'socialism').select(['author'])
df_submissions_liberal_users = df_submissions_socialism_economics.withColumn("created_utc", date_format("created_utc", "yyyy-MM-dd")).filter(df_submissions_socialism_economics['subreddit'] == 'Liberal').select(['author'])
df_submissions_askpol_users = df_submissions_askpol_cmv.withColumn("created_utc", date_format("created_utc", "yyyy-MM-dd")).filter(df_submissions_askpol_cmv['subreddit'] == 'Ask_Politics').select(['author'])
df_submissions_cmv_users = df_submissions_askpol_cmv.withColumn("created_utc", date_format("created_utc", "yyyy-MM-dd")).filter(df_submissions_askpol_cmv['subreddit'] == 'changemyview').select(['author'])

In [35]:
df_submissions_socialism_users.show()

+--------------------+
|              author|
+--------------------+
|        mindvarious2|
|          poohbadger|
|        pashotboshot|
|       EternityWatch|
|        pashotboshot|
|           [deleted]|
|       MarxistZaheer|
|          Thatsplumb|
|           Mellows22|
|Infamous_Apartment41|
|              rewkom|
|    Patterson9191717|
|        pashotboshot|
|              rewkom|
|           [deleted]|
|           [deleted]|
|        sillychillly|
|      Nick__________|
|           kalhoon01|
| Spartanwildcats2018|
+--------------------+
only showing top 20 rows



In [19]:
df_submissions_askpol_users.show()

+-------------------+
|             author|
+-------------------+
|  ModerateRockMusic|
|    The6thMessenger|
|     Fuligo_septica|
|       Saith_Cassus|
| Ok-Philosophy-5084|
|          [deleted]|
|          Scutarius|
|          [deleted]|
|          [deleted]|
|          [deleted]|
|    ThatAgnosticGuy|
|              Frumk|
|JUSTICEWILLRAINDOWN|
|          [deleted]|
|             KLombe|
|    lonely_Deer8609|
|       TwoCheap9240|
|            Markdd8|
|     Fuligo_septica|
|         Ent3rpris3|
+-------------------+
only showing top 20 rows



In [20]:
# Clean

df_submissions_liberterian_users = df_submissions_liberterian_users.filter(df_submissions_liberterian_users['author'] != "[deleted]")
df_submissions_centrist_users = df_submissions_centrist_users.filter(df_submissions_centrist_users['author'] != "[deleted]")
df_submissions_conservative_users = df_submissions_conservative_users.filter(df_submissions_conservative_users['author'] != "[deleted]")
df_submissions_socialism_users = df_submissions_socialism_users.filter(df_submissions_socialism_users['author'] != "[deleted]")
df_submissions_liberal_users = df_submissions_liberal_users.filter(df_submissions_liberal_users['author'] != "[deleted]")
df_submissions_askpol_users = df_submissions_askpol_users.filter(df_submissions_askpol_users['author'] != "[deleted]")
df_submissions_cmv_users = df_submissions_cmv_users.filter(df_submissions_cmv_users['author'] != "[deleted]")

In [21]:
# Remove duplicates

df_submissions_liberterian_users = df_submissions_liberterian_users.distinct()
df_submissions_centrist_users = df_submissions_centrist_users.distinct()
df_submissions_conservative_users = df_submissions_conservative_users.distinct()
df_submissions_socialism_users = df_submissions_socialism_users.distinct()
df_submissions_liberal_users = df_submissions_liberal_users.distinct()
df_submissions_askpol_users = df_submissions_askpol_users.distinct()
df_submissions_cmv_users = df_submissions_cmv_users.distinct()


In [52]:
df_submissions_liberterian_users.show()

+-------------------+
|             author|
+-------------------+
| Friendly-Risk-9422|
|           space700|
|  Wake-up-Neo-sheep|
| SilverKnightGundam|
|BrekfastLibertarian|
|    SuperSweetStuff|
|         TreginWork|
|           Joshau-k|
|      gbgghmre657u6|
|ConversationDue4320|
|           mrglass8|
|  airassault_tanker|
|          Jules0328|
|  TheLemonEater5000|
|  Fuckleberry__Finn|
|         AtlasRoark|
|      Steeler_Train|
|       rytron3000om|
|   ViolenceSolution|
|        tareklawyer|
+-------------------+
only showing top 20 rows



In [39]:
df_submissions_liberal_users.show()

+-------------------+
|             author|
+-------------------+
|            itcrygo|
|           Helpbrin|
|       nippleflick1|
|    jackbenimble111|
|   Minnesota__Scott|
|   GlacnerTheMighty|
|         progress18|
|    Andalib_Odulate|
|     medicatedincel|
|        MilesAlston|
|       nicholashuey|
|       PogSlamShady|
|    Uppersideofhell|
|          jmooremcc|
|    UnknownAuthor42|
|       SpicyBunny28|
|jesusgetsbuttfucked|
|   televisiontunnel|
|       matchettehdl|
|      flamingo23232|
+-------------------+
only showing top 20 rows



In [23]:
df_submissions_cmv_users.show()

+--------------------+
|              author|
+--------------------+
|              a1996p|
|          BanksRScam|
|      dmkicksballs13|
|          Mad_Prog_1|
|        stilltilting|
|        They_took_it|
|            pragmojo|
|         coinboi2012|
|       TripleABeauty|
|            poopspec|
|            FarIdiom|
|        Mad_Chemist_|
|    Rough_Spirit4528|
|         Ohm-Abc-123|
|      throwaway-9844|
|Affectionate_Chair15|
|             rebsar1|
|           Blackoffi|
|       IdesOfCaesar7|
|      trainwreck7775|
+--------------------+
only showing top 20 rows



In [40]:
# Cell 1
common_users_lib_liberal = df_submissions_liberterian_users.join(df_submissions_liberal_users, "author")
num_common_users_lib_liberal = common_users_lib_liberal.count()
print(f"Common users between Libertarian and Liberal: {num_common_users_lib_liberal}")


Common users between Libertarian and Liberal: 426


In [41]:
# Cell 2
common_users_centrist_conservative = df_submissions_centrist_users.join(df_submissions_conservative_users, "author")
num_common_users_centrist_conservative = common_users_centrist_conservative.count()
print(f"Common users between Centrist and Conservative: {num_common_users_centrist_conservative}")


Common users between Centrist and Conservative: 378


In [42]:
# Cell 3
common_users_centrist_liberal = df_submissions_centrist_users.join(df_submissions_liberal_users, "author")
num_common_users_centrist_liberal = common_users_centrist_liberal.count()
print(f"Common users between Centrist and Liberal: {num_common_users_centrist_liberal}")


Common users between Centrist and Liberal: 93


In [43]:
# Cell 4
common_users_conservative_liberal = df_submissions_conservative_users.join(df_submissions_liberal_users, "author")
num_common_users_conservative_liberal = common_users_conservative_liberal.count()
print(f"Common users between Conservative and Liberal: {num_common_users_conservative_liberal}")


Common users between Conservative and Liberal: 815


In [44]:
# Cell 5
common_users_conservative_socialism = df_submissions_conservative_users.join(df_submissions_socialism_users, "author")
num_common_users_conservative_socialism = common_users_conservative_socialism.count()
print(f"Common users between Conservative and Socialism: {num_common_users_conservative_socialism}")


Common users between Conservative and Socialism: 863


In [45]:
# Cell 6
common_users_liberal_socialism = df_submissions_liberal_users.join(df_submissions_socialism_users, "author")
num_common_users_liberal_socialism = common_users_liberal_socialism.count()
print(f"Common users between Liberal and Socialism: {num_common_users_liberal_socialism}")


Common users between Liberal and Socialism: 293


In [24]:
# Define a function to find common users and return the count
def count_common_users(df1, df1_name, df2, df2_name):
    common_users = df1.join(df2, "author")
    num_common_users = common_users.count()
    return [df1_name, df2_name, num_common_users]

# List to store the output
common_users_counts = []

# Calculate the number of common users between each pair of subreddits
common_users_counts.append(count_common_users(df_submissions_liberterian_users, 'Libertarian', df_submissions_centrist_users, 'Centrist'))
common_users_counts.append(count_common_users(df_submissions_liberterian_users, 'Libertarian', df_submissions_conservative_users, 'Conservative'))
common_users_counts.append(count_common_users(df_submissions_liberterian_users, 'Libertarian', df_submissions_socialism_users, 'Socialism'))
common_users_counts.append(count_common_users(df_submissions_liberterian_users, 'Libertarian', df_submissions_liberal_users, 'Liberal'))
common_users_counts.append(count_common_users(df_submissions_centrist_users, 'Centrist', df_submissions_conservative_users, 'Conservative'))
common_users_counts.append(count_common_users(df_submissions_centrist_users, 'Centrist', df_submissions_socialism_users, 'Socialism'))
common_users_counts.append(count_common_users(df_submissions_centrist_users, 'Centrist', df_submissions_liberal_users, 'Liberal'))
common_users_counts.append(count_common_users(df_submissions_conservative_users, 'Conservative', df_submissions_socialism_users, 'Socialism'))
common_users_counts.append(count_common_users(df_submissions_conservative_users, 'Conservative', df_submissions_liberal_users, 'Liberal'))
common_users_counts.append(count_common_users(df_submissions_socialism_users, 'Socialism', df_submissions_liberal_users, 'Liberal'))

# Now print the list in the desired format
print(common_users_counts)


[['Libertarian', 'Centrist', 274], ['Libertarian', 'Conservative', 2494], ['Libertarian', 'Socialism', 648], ['Libertarian', 'Liberal', 426], ['Centrist', 'Conservative', 378], ['Centrist', 'Socialism', 82], ['Centrist', 'Liberal', 93], ['Conservative', 'Socialism', 863], ['Conservative', 'Liberal', 815], ['Socialism', 'Liberal', 293]]


In [26]:
# Add the remainining calls

# Append the comparisons with Ask_Politics and ChangeMyView
common_users_counts.append(count_common_users(df_submissions_liberterian_users, 'Libertarian', df_submissions_askpol_users, 'Ask_Politics'))
common_users_counts.append(count_common_users(df_submissions_liberterian_users, 'Libertarian', df_submissions_cmv_users, 'ChangeMyView'))
common_users_counts.append(count_common_users(df_submissions_centrist_users, 'Centrist', df_submissions_askpol_users, 'Ask_Politics'))
common_users_counts.append(count_common_users(df_submissions_centrist_users, 'Centrist', df_submissions_cmv_users, 'ChangeMyView'))
common_users_counts.append(count_common_users(df_submissions_conservative_users, 'Conservative', df_submissions_askpol_users, 'Ask_Politics'))
common_users_counts.append(count_common_users(df_submissions_conservative_users, 'Conservative', df_submissions_cmv_users, 'ChangeMyView'))
common_users_counts.append(count_common_users(df_submissions_socialism_users, 'Socialism', df_submissions_askpol_users, 'Ask_Politics'))
common_users_counts.append(count_common_users(df_submissions_socialism_users, 'Socialism', df_submissions_cmv_users, 'ChangeMyView'))
common_users_counts.append(count_common_users(df_submissions_liberal_users, 'Liberal', df_submissions_askpol_users, 'Ask_Politics'))
common_users_counts.append(count_common_users(df_submissions_liberal_users, 'Liberal', df_submissions_cmv_users, 'ChangeMyView'))
common_users_counts.append(count_common_users(df_submissions_askpol_users, 'Ask_Politics', df_submissions_cmv_users, 'ChangeMyView'))


In [27]:
common_users_counts

[['Libertarian', 'Centrist', 274],
 ['Libertarian', 'Conservative', 2494],
 ['Libertarian', 'Socialism', 648],
 ['Libertarian', 'Liberal', 426],
 ['Centrist', 'Conservative', 378],
 ['Centrist', 'Socialism', 82],
 ['Centrist', 'Liberal', 93],
 ['Conservative', 'Socialism', 863],
 ['Conservative', 'Liberal', 815],
 ['Socialism', 'Liberal', 293],
 ['Libertarian', 'Ask_Politics', 124],
 ['Libertarian', 'ChangeMyView', 446],
 ['Centrist', 'Ask_Politics', 47],
 ['Centrist', 'ChangeMyView', 170],
 ['Conservative', 'Ask_Politics', 208],
 ['Conservative', 'ChangeMyView', 880],
 ['Socialism', 'Ask_Politics', 93],
 ['Socialism', 'ChangeMyView', 266],
 ['Liberal', 'Ask_Politics', 50],
 ['Liberal', 'ChangeMyView', 126],
 ['Ask_Politics', 'ChangeMyView', 190]]

## Bubble Chart

In [51]:
df_submissions_liberterian = df_submissions_centrist_liberterian.withColumn("created_utc", date_format("created_utc", "yyyy-MM-dd")).filter(df_submissions_centrist_liberterian['subreddit'] == 'Libertarian').select(['created_utc', 'subreddit' , 'title' , 'url' , 'score'])
df_submissions_centrist = df_submissions_centrist_liberterian.withColumn("created_utc", date_format("created_utc", "yyyy-MM-dd")).filter(df_submissions_centrist_liberterian['subreddit'] == 'centrist').select(['created_utc', 'subreddit' , 'title', 'url', 'score'])
df_submissions_liberal = df_submissions_socialism_economics.withColumn("created_utc", date_format("created_utc", "yyyy-MM-dd")).filter(df_submissions_socialism_economics['subreddit'] == 'Liberal').select(['created_utc', 'subreddit', 'title', 'url', 'score'])
df_submissions_conservative = df_submissions_conservative_finance.withColumn("created_utc", date_format("created_utc", "yyyy-MM-dd")).filter(df_submissions_conservative_finance['subreddit'] == 'Conservative').select(['created_utc', 'subreddit', 'title', 'url', 'score'])
df_submissions_socialism = df_submissions_socialism_economics.withColumn("created_utc", date_format("created_utc", "yyyy-MM-dd")).filter(df_submissions_socialism_economics['subreddit'] == 'socialism').select(['created_utc', 'subreddit', 'title', 'url', 'score'])

In [52]:
from functools import reduce
from pyspark.sql import DataFrame

# Combine all the DataFrames into one using union
df_combined = df_submissions_liberterian \
    .union(df_submissions_centrist) \
    .union(df_submissions_liberal) \
    .union(df_submissions_conservative) \
    .union(df_submissions_socialism)

In [53]:
df_combined.show()

+-----------+-----------+--------------------+--------------------+-----+
|created_utc|  subreddit|               title|                 url|score|
+-----------+-----------+--------------------+--------------------+-----+
| 2021-09-06|Libertarian|I swear, people p...|https://www.reddi...|   38|
| 2021-09-06|Libertarian|Is it abortion bl...|https://www.reddi...|    0|
| 2021-06-13|Libertarian|What are your tho...|https://www.reddi...|    0|
| 2021-06-13|Libertarian|9 Reasons Why Fed...|https://www.daily...|    5|
| 2021-05-09|Libertarian|Economics and Ethics|https://the-mindf...|    7|
| 2021-05-09|Libertarian|Great and inspiri...|https://www.youtu...|    0|
| 2021-05-09|Libertarian|Social Media Cens...|https://www.reddi...|    2|
| 2021-08-06|Libertarian|Is Ron Desantis a...|https://www.reddi...|    0|
| 2021-08-06|Libertarian|'Buy America' Was...|https://www.bloom...|   12|
| 2021-08-06|Libertarian|State Con Law Cas...|https://ij.org/cj...|   10|
| 2021-08-06|Libertarian|More people n

In [55]:
 # Keep highest 50 posts per subreddit (highest in terms of score)
    
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

# Define the window specification
windowSpec = Window.partitionBy("subreddit").orderBy(df_combined["score"].desc())

# Add row number for each row within each partition
df_with_row_number = df_combined.withColumn("row_number", row_number().over(windowSpec))

# Filter for top 50 posts per subreddit
top_50_per_subreddit = df_with_row_number.filter(df_with_row_number["row_number"] <= 50)

# Now you can drop the row_number column if you don't need it anymore
top_50_per_subreddit = top_50_per_subreddit.drop("row_number")

# Show the result
top_50_per_subreddit.show()


+-----------+---------+--------------------+--------------------+-----+
|created_utc|subreddit|               title|                 url|score|
+-----------+---------+--------------------+--------------------+-----+
| 2021-01-27|  Liberal|Republicans now '...|https://www.msn.c...| 1127|
| 2022-05-11|  Liberal|Pro-Trump Electio...|https://www.bloom...| 1074|
| 2022-08-10|  Liberal|Most Americans su...|https://www.npr.o...|  969|
| 2021-01-11|  Liberal|Report: QAnon Con...|https://www.pathe...|  967|
| 2021-01-30|  Liberal|GOP group launche...|https://www.indep...|  922|
| 2021-01-28|  Liberal|Joe Biden’s Done ...|https://www.theda...|  912|
| 2023-03-31|  Liberal|'I reported 7 to ...|                    |  898|
| 2022-11-09|  Liberal|Trump Reportedly ...|https://people.co...|  887|
| 2021-09-04|  Liberal|Oh My Fucking God...|https://www.mcswe...|  855|
| 2022-05-27|  Liberal|The Republican Pa...|https://www.msnbc...|  846|
| 2021-08-02|  Liberal|Rudy Giuliani Is ...|https://www.mothe...

In [56]:
# Convert to pandas for ease of use

# Ensure that the Spark DataFrame is small enough to fit into the local memory
top_50_per_subreddit_pandas = top_50_per_subreddit.toPandas()

# Now you have the DataFrame in pandas format
top_50_per_subreddit_pandas


,created_utc,subreddit,title,url,score
0,2021-01-27,Liberal,"Republicans now 'shocked, shocked' that there'...",https://www.msn.com/en-us/news/politics/republ...,1127
1,2022-05-11,Liberal,Pro-Trump Election Deniers Lose in Nebraska Re...,https://www.bloomberg.com/news/articles/2022-0...,1074
2,2022-08-10,Liberal,Most Americans support using the popular vote ...,https://www.npr.org/2022/08/10/1116688726/most...,969
3,2021-01-11,Liberal,Report: QAnon Congresswoman Was Live-Tweeting ...,https://www.patheos.com/blogs/progressivesecul...,967
4,2021-01-30,Liberal,GOP group launches billboards demanding Cruz a...,https://www.independent.co.uk/news/world/ameri...,922
...,...,...,...,...,...
245,2021-05-13,Conservative,So this is acceptable,https://i.redd.it/x5rmktghgyy61.png,12209
246,2021-12-23,Conservative,Nancy Pelosi s Husband Seems to be a Savant at...,https://politics.retalk.com/nancy-pelosis-husb...,12133
247,2022-02-24,Conservative,BREAKING Russia Begins Full Scale Invasion In...,https://www.dailywire.com/news/breaking-russia...,11981
248,2021-01-08,Conservative,Reddit bans r Donaldtrump for repeated violati...,https://www.cnet.com/news/reddit-bans-rdonaldt...,11614


In [61]:
top_50_per_subreddit_pandas.head()

,created_utc,subreddit,title,url,score
0,2021-01-27,Liberal,"Republicans now 'shocked, shocked' that there'...",https://www.msn.com/en-us/news/politics/republ...,1127
1,2022-05-11,Liberal,Pro-Trump Election Deniers Lose in Nebraska Re...,https://www.bloomberg.com/news/articles/2022-0...,1074
2,2022-08-10,Liberal,Most Americans support using the popular vote ...,https://www.npr.org/2022/08/10/1116688726/most...,969
3,2021-01-11,Liberal,Report: QAnon Congresswoman Was Live-Tweeting ...,https://www.patheos.com/blogs/progressivesecul...,967
4,2021-01-30,Liberal,GOP group launches billboards demanding Cruz a...,https://www.independent.co.uk/news/world/ameri...,922


In [64]:
top_50_per_subreddit_pandas['subreddit'].value_counts()

Liberal         50
Libertarian     50
centrist        50
socialism       50
Conservative    50
Name: subreddit, dtype: int64

In [75]:
import pandas as pd

# Assume df is your DataFrame
# Convert the DataFrame to the desired JSON-like list of dictionaries format
def transform_dataframe(df):
    result = []
    # Group the DataFrame by subreddit and iterate over each group
    for subreddit, group in df.groupby('subreddit'):
        subreddit_data = {
            'name': subreddit,
            'data': [
                {
                    'name': title,
                    'value': score,
                    'score': score, # This line adds the score
                    'url': url      # This line adds the url
                } for title, score, url in zip(group['title'], group['score'], group['url']) # Ensure you are pulling the url here as well
            ]
        }
        result.append(subreddit_data)
    return result

# Assuming top_50_per_subreddit_pandas is your DataFrame name
json_like_structure = transform_dataframe(top_50_per_subreddit_pandas)


In [76]:
json_like_structure

[{'name': 'Conservative',
  'data': [{'name': 'For those of you comparing these protests to Boston 1773  take a look at these pictures and tell me how this is patriotic to you  That is The Capitol Building',
    'value': 110685,
    'score': 110685,
    'url': 'https://www.reddit.com/gallery/krxl6t'},
   {'name': 'Fox News  McConnell believes Trump committed impeachable offenses  supports Democrats  impeachment efforts',
    'value': 48994,
    'score': 48994,
    'url': 'https://www.foxnews.com/politics/mcconnell-believes-trump-committed-impeachable-offenses-supports-democrats-impeachment-efforts-report'},
   {'name': 'House impeaches Trump for second time over Capitol riots',
    'value': 45677,
    'score': 45677,
    'url': 'http://www.breakingthenews.net/news/details/54323954'},
   {'name': 'New SEC Rule  Wall Street Will Now Only Allow Traders Who Wear A Top Hat And Monocle And Carry Around Giant Bags Of Money',
    'value': 44190,
    'score': 44190,
    'url': 'https://babylonb

In [78]:
top_50_per_subreddit_pandas['url'].value_counts()

https://www.usasupreme.com/biden-voter-on-cnn-theyre-dropping-bombs-in-syria-and-those-bombs-are-pretty-expensive-for-a-guy-who-owes-me-2000-video/                                2
https://v.redd.it/wb5brhtlfen91                                                                                                                                                    1
https://i.redd.it/vbhgnfoi62h81.png                                                                                                                                                1
https://v.redd.it/sgslc4vjm0i81                                                                                                                                                    1
https://i.redd.it/fyjlka3rhf961.jpg                                                                                                                                                1
                                                                                               